In [1]:
import torch
from transformers import MobileNetV2Config, MobileNetV2Model
import torchaudio
import iree.turbine.aot as aot

/home/gilro/ml_stuff/infra/iree/venv_online_everything/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = MobileNetV2Config()
model = MobileNetV2Model(config)
sample_inputs = torch.randn((1, 3, 224, 224))
print(f"{model(sample_inputs).last_hidden_state.shape=}")


class Wrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x):
        return self.model(x).last_hidden_state


model_ = Wrapper(model).cpu().eval()
export_output = aot.export(model_, sample_inputs)
export_output.save_mlir("converted_mobilenetv2_default_config.mlir")

model(sample_inputs).last_hidden_state.shape=torch.Size([1, 1280, 7, 7])


/home/gilro/ml_stuff/infra/iree/venv_online_everything/lib/python3.12/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [4]:
sum([p.numel() for p in model.parameters()]) * 4

8895488